In [ ]:
using Random
using Images
using FileIO

# Палитра цветов радуги
function get_rainbow_palette()
    return [
        RGB(1.0, 0.0, 0.0),     # красный
        RGB(1.0, 0.5, 0.0),     # оранжевый
        RGB(1.0, 1.0, 0.0),     # желтый
        RGB(0.0, 1.0, 0.0),     # зеленый
        RGB(0.0, 1.0, 1.0),     # голубой
        RGB(0.0, 0.0, 1.0),     # синий
        RGB(0.5, 0.0, 1.0)      # фиолетовый
    ]
end

# Преобразование матрицы в изображение
function matrix_to_image(matrix, palette)
    h, w = size(matrix)
    img = Array{RGB{Float64}}(undef, h, w)
    for i in 1:h
        for j in 1:w
            idx = clamp(matrix[i, j], 1, length(palette))
            img[i, j] = palette[idx]
        end
    end
    return img
end

# Преобразование матрицы в бинарный формат
function matrix_to_binary(matrix, bits_per_element)
    binary_vec = Bool[]
    for elem in matrix
        val = elem - 1
        bin_str = bitstring(val)[end-2:end] # 3 бита
        append!(binary_vec, [c == '1' for c in bin_str])
    end
    return binary_vec
end

# Преобразование бинарного формата в матрицу
function binary_to_matrix(binary_vec, bits_per_element, matrix_size)
    matrix = zeros(Int, matrix_size)
    for i in 1:prod(matrix_size)
        bits = binary_vec[(i-1)*bits_per_element+1:i*bits_per_element]
        val = sum(bits .* [4, 2, 1]) + 1
        matrix[i] = clamp(val, 1, 7)
    end
    return matrix
end

# Кроссовер (равномерный)
function crossover(a, b)
    child = similar(a)
    for i in eachindex(a)
        child[i] = rand() < 0.5 ? a[i] : b[i]
    end
    return child
end

# Мутация
function mutate(binary_vec, mutation_rate)
    mutated = copy(binary_vec)
    for i in eachindex(mutated)
        rand() < mutation_rate && (mutated[i] = !mutated[i])
    end
    return mutated
end

# Генетический алгоритм
function genetic_algorithm(;
        matrix_size = (9, 9),
        epsilon = 0.1,
        max_generations = 1000,
        mutation_rate = 0.01,
        use_hamming = true)
    
    palette = get_rainbow_palette()
    bits_per_element = 3
    
    # Генерация эталонной матрицы
    reference = rand(1:7, matrix_size)
    current = rand(1:7, matrix_size)
    
    # Сохранение кадров для гифки
    frames = [matrix_to_image(current, palette)]
    
    # Преобразование в бинарный формат
    ref_bin = matrix_to_binary(reference, bits_per_element)
    current_bin = matrix_to_binary(current, bits_per_element)
    
    generation = 0
    while generation < max_generations
        # Создание нового поколения
        child_bin = crossover(current_bin, ref_bin)
        child_bin = mutate(child_bin, mutation_rate)
        
        # Обратное преобразование
        child = binary_to_matrix(child_bin, bits_per_element, matrix_size)
        
        # Расчет расстояния
        if use_hamming
            dist = sum(current .!= reference)
            new_dist = sum(child .!= reference)
        else
            dist = sqrt(sum((current .- reference).^2))
            new_dist = sqrt(sum((child .- reference).^2))
        end
        
        # Выбор лучшего решения
        if new_dist < dist
            current = child
            current_bin = child_bin
            push!(frames, matrix_to_image(current, palette))
        end
        
        # Проверка условия останова
        (new_dist < epsilon) && break
        generation += 1
    end
    
    # Сохранение результатов
    save("reference.png", matrix_to_image(reference, palette))
    save("result.png", matrix_to_image(current, palette))
    save("evolution.gif", cat(frames..., dims=3), fps=10)
    
    return reference, current
end

# Запуск алгоритма с параметрами по умолчанию
genetic_algorithm(use_hamming=false, epsilon=0.5, max_generations=500)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


([5 2 … 5 4; 4 7 … 2 7; … ; 1 5 … 6 2; 4 2 … 6 7], [5 2 … 5 4; 4 7 … 2 7; … ; 1 5 … 6 2; 4 2 … 6 7])